In [4]:
from __future__ import division, absolute_import
import pandas as pd
import numpy as np
import datetime as dt
import forecastio
import requests
import os
import csv
from generate_data import get_geolocation_data
from helpers import get_filepath, daterange, save_data

source = 'locations.txt'
target = 'geocoded_locations.txt'
get_geolocation_data(source, target)
api_key = '63ab81b2d8aee963f8e0c22cd4ec4650'

In [23]:
start_date = dt.datetime(2017, 1, 1)
end_date = dt.datetime(2017, 1, 2)
locs = 'geocoded_locations.txt'
cols = 'columns.txt'

locs_path = get_filepath(locs)
locs = pd.read_csv(locs_path)

# get columns list
columns = get_filepath(cols)
with open(columns) as f:
    cols = [line.strip() for line in f]

# extract data for each location for date range b/w start and end date
for index, row in locs.iterrows():
    tbl = []
    for single_date in daterange(start_date, end_date):
        forecast = forecastio.load_forecast(api_key,
                                            row['lat'],
                                            row['lng'],
                                            time=single_date,
                                            units='si')
        h = forecast.hourly()
        d = h.data
        for p in d:
            # get date info
            utc = p.d['time']
            dts = dt.datetime.utcfromtimestamp(utc)
            isodate = dt.datetime.utcfromtimestamp(utc).isoformat()
            date_info = [isodate, dts.year, dts.month, dts.day, dts.hour]

            # get location info
            loc_info = [row['location'], row['lat'], row['lng']]

            # get weather attributes
            attr_info = [p.d['icon'], p.d['precipProbability'], p.d['temperature'], p.d['dewPoint'], p.d['humidity'] * 100, p.d['pressure'], p.d['windSpeed'], p.d['visibility']]
            tbl.append(loc_info + date_info + attr_info)

    # convert output to data frame
    df = pd.DataFrame(tbl)
    df.columns = cols
    df.name = row['location']
    sd_str = str(start_date.date())
    ed_str = str(end_date.date())
    filename = '_'.join([df.name, sd_str, ed_str]) + '.csv'
    save_data(df, filename, sep='|')

In [ ]:
          "time": 1509991200,
                  "summary": "Mostly Cloudy",
                  "icon": "partly-cloudy-day",
                  "precipIntensity": 0.0007,
                  "precipProbability": 0.1,
                  "precipType": "rain",
                  "temperature": 65.76,
                  "apparentTemperature": 66.01,
                  "dewPoint": 60.99,
                  "humidity": 0.85,
                  "pressure": 1010.57,
                  "windSpeed": 4.23,
                  "windGust": 9.52,
                  "windBearing": 230,
                  "cloudCover": 0.62,
                  "uvIndex": 1,
                  "visibility": 9.32,
                  "ozone": 268.95